# Filtering .vcf file from ipyrad

This notebook details the secondary filtering I do with a .vcf file of SNPs directly from `ipyrad`. I use this notebook to:  
1. filter out SNPs missing in > 25% of samples (`ipyrad` does this on a locus basis, but with indels and *N*s there could still be sites that are missing in a lot of samples), 
2. filter for only biallelic SNPs,
3. filter out GBS loci containing SNPs with excess heterozygosity based on Hardy-Weinberg equilibrium in at least 2 populations, 
4. filter out SNPs with a minor allele frequency < 2.5%, and 
5. subset one SNP per GBS locus for analyses requiring unlinked loci.  

Pay close attention to file paths if attempting to rerun this notebook, as they may be different on your computer. The directory structure used in this notebook assumes that you have downloaded the directory of output files produced by `ipyrad` from [Dryad](https://datadryad.org/resource/doi:10.5061/dryad.114j8m1/6) and unzipped it, so that you have a folder called `OL-s7filt45-c85-t10_outfiles`. Make sure to note the location of where you download this directory, as it will be required as input in the first code chunk. Most code chunks starts with `suffix=OL-c85t10-x45m75`. This suffix is added to the beginning of output files, and can be changed if desired. 

**Note:** I set the *max_shared_Hs_locus* parameter in `ipyrad` to 1.0 so it does not filter for excess heterozygotes across samples. 

## Filtering .vcf from *ipyrad*

Use [VCFtools](https://vcftools.github.io/man_latest.html) to filter for polymorphic, biallelic loci and loci found in at least 75% of individuals. This is the full SNP dataset, before filtering for minor allele frequency and Hardy-Weinberg equilibrium.

In [2]:
%%sh
suffix=OL-c85t10-x45m75
## location of VCF file from ipyrad
ipVCF=../Assembly/OL-s7filt45-c85-t10_outfiles/OL-s7filt45-c85-t10.vcf

vcftools --vcf ${ipVCF} --recode --recode-INFO-all \
--min-alleles 2 --max-alleles 2 \
--max-missing 0.75 \
--out ${suffix}


VCFtools - 0.1.15
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf ../Assembly/OL-s7filt45-c85-t10_outfiles/OL-s7filt45-c85-t10.vcf
	--recode-INFO-all
	--max-alleles 2
	--min-alleles 2
	--max-missing 0.75
	--out OL-c85t10-x45m75
	--recode

After filtering, kept 117 out of 117 Individuals
Outputting VCF file...
After filtering, kept 42118 out of a possible 58963 Sites
Run Time = 5.00 seconds


Create a file listing the number and proportion of SNPs missing in each individual. Also used as a list of individuals found in the file.

In [3]:
%%sh
suffix=OL-c85t10-x45m75
vcftools --vcf $suffix.recode.vcf --missing-indv --out $suffix


VCFtools - 0.1.15
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf OL-c85t10-x45m75.recode.vcf
	--missing-indv
	--out OL-c85t10-x45m75

After filtering, kept 117 out of 117 Individuals
Outputting Individual Missingness
After filtering, kept 42118 out of a possible 42118 Sites
Run Time = 1.00 seconds


Write files with the sample name and either population (.pop) or all strata info (.strata). These are used for the heterozygosity filtering downstream and to convert the .vcf file to other file formats with the *radiator* package in R. It is dependent on the samples having their population coded as the first part of their name, then separated by an underscore. It uses the .imiss file created earlier in this notebook.  

The .strata file is also available on [Dryad](https://datadryad.org/resource/doi:10.5061/dryad.114j8m1/5).

In [5]:
IN = open("OL-c85t10-x45m75.imiss","r")
OUT_strata = open("OL-c85t10-x45.strata","w")
OUT_pop = open("OL-c85t10-x45.pop","w")
loc_dict = {'BC1':['Victoria_BC', 'Victoria_BC', 'Puget+BC', '48.435667', '-123.377909'],
            'BC2':['Klaskino_BC', 'Klaskino_BC', 'NWBC', '50.298667', '-127.723633'],
            'BC3':['Barkley_BC', 'Barkley_BC', 'NWBC', '49.01585', '-125.314167'],
            'BC4': ['Ladysmith_BC', 'Ladysmith_BC', 'Puget+BC','49.011383' ,'-123.8357'],
            'WA12': ['TritonCove_WA', 'TritonCove_WA', 'Puget+BC', '47.6131', '-122.982'],
            'WA11': ['Liberty_WA', 'Liberty_WA', 'Puget+BC', '47.7375', '-122.6507'],
            'WA13': ['NorthBay_WA', 'NorthBay_WA', 'Puget+BC','47.3925', '-122.8138'],
            'WA10': ['Discovery_WA', 'Discovery_WA', 'Puget+BC', '47.9978', '-122.8824'],
            'WA1': ['Willapa_WA', 'NorthWillapa_WA', 'Willapa', '46.624772', '-123.9887916'],
            'WA9': ['Willapa_WA', 'SouthWillapa_WA', 'Willapa', '46.44', '-124.004'],
            'OR3': ['Netarts_OR', 'Netarts_OR', 'Oregon', '45.3911556', '-123.9559028'],
            'OR2': ['Yaquina_OR', 'Yaquina_OR', 'Oregon', '44.579539', '-123.995749'],
            'OR1': ['Coos_OR', 'Coos_OR', 'Willapa', '43.3559861', '-124.1931639'],
            'CA6': ['Humboldt_CA', 'Humboldt_CA', 'NoCal', '40.8557972', '-124.0974611'],
            'CA4': ['Tomales_CA', 'Tomales_CA', 'NoCal', '38.117549', '-122.874497'],
            'CA2': ['NorthSanFran_CA', 'NorthSanFran_CA', 'NoCal', '37.955067', '-122.421800'],
            'CA3': ['SouthSanFran_CA', 'SouthSanFran_CA', 'NoCal', '37.708665', '-122.377607'],
            'CA5': ['Elkhorn_CA', 'Elkhorn_CA', 'SoCal', '36.8398194', '-121.7427806'],
            'CA7': ['MuguLagoon_CA', 'MuguLagoon_CA', 'SoCal', '34.101914', '-119.10434'],
            'CA1': ['SanDiego_CA', 'SanDiego_CA', 'SoCal', '32.602500', '-117.118889']}
IN.next()
OUT_strata.write("INDIVIDUALS\tSTRATA\tLOCATION\tREGION\tLATITUDE\tLONGITUDE\tLIBRARY\n")
for line in IN:
    name = line.split()[0]
    pop = name.split("_")[0]
    library = name.split("_")[2]
    OUT_strata.write(name+"\t"+'\t'.join(map(str,loc_dict[pop]))+"\t"+library+"\n")
    OUT_pop.write(name+"\t"+loc_dict[pop][0]+"\n")
    
IN.close()
OUT_strata.close()
OUT_pop.close()

### Filtering loci by departures from Hardy-Weinberg

Here I filter out loci that depart from Hardy-Weinberg equilibrium in at least 2 populations with a p-value cutoff of 0.05. It takes a .vcf file and the .pop file just created as input. This uses a script from [Jon Puritz's Github](https://github.com/jpuritz/dDocent/blob/master/scripts/filter_hwe_by_pop.pl), written by Chris Hollenbeck. 

In [7]:
%%sh
suffix=OL-c85t10-x45m75
## Filtering out loci that depart HWE in at least 2 populations with a p-value cutoff of 0.05
../Scripts/filter_hwe_by_pop.pl -v $suffix.recode.vcf \
-p OL-c85t10-x45.pop -h 0.05 -c 0.09 -o $suffix-hwPbi

mv exclude.hwe exclude.txt
## remove intermediate files
rm *.inds
## Remove these if you don't want to inspect HWE results
rm *.hwe

Processing population: Barkley_BC (5 inds)
Processing population: Coos_OR (6 inds)
Processing population: Discovery_WA (7 inds)
Processing population: Elkhorn_CA (6 inds)
Processing population: Humboldt_CA (6 inds)
Processing population: Klaskino_BC (8 inds)
Processing population: Ladysmith_BC (5 inds)
Processing population: Liberty_WA (6 inds)
Processing population: MuguLagoon_CA (9 inds)
Processing population: Netarts_OR (7 inds)
Processing population: NorthBay_WA (6 inds)
Processing population: NorthSanFran_CA (5 inds)
Processing population: SanDiego_CA (7 inds)
Processing population: SouthSanFran_CA (4 inds)
Processing population: Tomales_CA (6 inds)
Processing population: TritonCove_WA (6 inds)
Processing population: Victoria_BC (7 inds)
Processing population: Willapa_WA (5 inds)
Processing population: Yaquina_OR (6 inds)
Outputting results of HWE test for filtered loci to 'filtered.hwe'
Kept 41924 of a possible 42118 loci (filtered 194 loci)


The script only filters on a site-by-site basis. In order to throw out any GBS loci that had a SNP out of HWE (as these may be paralogs), I make a file with the locus ids to then submit to VCFtools.

In [8]:
## Make files of bad loci (that have at least one site out of HWE) to read in to in vcftools. 
IN = open('exclude.txt', "r")
OUT = open('badchrom.txt', "w")
exset = set()
for line in IN:
    chrom = line.split()[0]
    if chrom not in exset:
        exset.add(chrom)
        OUT.write(" --not-chr "+str(chrom))
OUT.close()
IN.close()
print "Loci to be filtered "+str(len(exset))

Loci to be filtered 149


The next code chunk will create a .vcf file that has just been filtered for missing data, biallelic SNPs, and HWE. This is not used in the analyses presented in the paper, so could be skipped if desired. First I make a folder called `Inputs/` to store all filtered files and subsequent input files for analyses.

In [9]:
%%sh
mkdir Inputs
## -filt are all SNPs, not filtering for minor allele frequency
value=`cat badchrom.txt`
suffix=OL-c85t10-x45m75

vcftools --vcf $suffix-hwPbi.recode.vcf --recode \
--recode-INFO-all $value --max-alleles 2 --min-alleles 2 --out Inputs/$suffix-filt


VCFtools - 0.1.15
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf OL-c85t10-x45m75-hwPbi.recode.vcf
	--not-chr locus_101729
	--not-chr locus_105574
	--not-chr locus_105872
	--not-chr locus_108364
	--not-chr locus_11229
	--not-chr locus_11344
	--not-chr locus_122277
	--not-chr locus_123093
	--not-chr locus_124336
	--not-chr locus_126680
	--not-chr locus_13078
	--not-chr locus_135077
	--not-chr locus_139738
	--not-chr locus_145304
	--not-chr locus_147286
	--not-chr locus_152226
	--not-chr locus_156530
	--not-chr locus_156836
	--not-chr locus_15739
	--not-chr locus_157656
	--not-chr locus_164593
	--not-chr locus_168485
	--not-chr locus_17009
	--not-chr locus_17236
	--not-chr locus_173735
	--not-chr locus_17649
	--not-chr locus_183021
	--not-chr locus_183515
	--not-chr locus_183542
	--not-chr locus_184001
	--not-chr locus_185388
	--not-chr locus_185910
	--not-chr locus_186505
	--not-chr locus_186710
	--not-chr locus_19446
	--not-chr locus_194493
	--not-chr loc

The next code chunk will create a .vcf file that has just been filtered for missing data, biallelic SNPs, HWE, and a minor allele frequency of 2.5%. This file is used to create input files for subsequent analyses presented in the paper.

In [11]:
%%sh
## -maf025 are SNPs after filtering for minor allele frequency of 2.5%
value=`cat badchrom.txt`
suffix=OL-c85t10-x45m75

vcftools --vcf $suffix-hwPbi.recode.vcf --recode --recode-INFO-all \
$value --max-alleles 2 --min-alleles 2 --maf 0.025 --out Inputs/$suffix-maf025


VCFtools - 0.1.15
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf OL-c85t10-x45m75-hwPbi.recode.vcf
	--not-chr locus_101729
	--not-chr locus_105574
	--not-chr locus_105872
	--not-chr locus_108364
	--not-chr locus_11229
	--not-chr locus_11344
	--not-chr locus_122277
	--not-chr locus_123093
	--not-chr locus_124336
	--not-chr locus_126680
	--not-chr locus_13078
	--not-chr locus_135077
	--not-chr locus_139738
	--not-chr locus_145304
	--not-chr locus_147286
	--not-chr locus_152226
	--not-chr locus_156530
	--not-chr locus_156836
	--not-chr locus_15739
	--not-chr locus_157656
	--not-chr locus_164593
	--not-chr locus_168485
	--not-chr locus_17009
	--not-chr locus_17236
	--not-chr locus_173735
	--not-chr locus_17649
	--not-chr locus_183021
	--not-chr locus_183515
	--not-chr locus_183542
	--not-chr locus_184001
	--not-chr locus_185388
	--not-chr locus_185910
	--not-chr locus_186505
	--not-chr locus_186710
	--not-chr locus_19446
	--not-chr locus_194493
	--not-chr loc

## Subset one SNP per GBS locus
Some analyses, like PCA and TreeMix, require unlinked loci. To try and meet this requirement, I create a .vcf file with only 1 SNP per GBS locus. 

In [12]:
## Code to subset one SNP per GBS locus from a VCF file. Chooses the SNP
## with the highest sample coverage. If there is a tie, chooses the 1st SNP in the loci. 
## May be specific to VCF format output from ipyrad.
## This is also in script format in Github as subsetSNPs.py

## With help from Michael A. Alcorn
def subsetSNPs(inputfile, outputfile):
    IN = open(inputfile, "r")
    OUT = open(outputfile, "w")

    current_locus = None
    best_NS = 0
    best_line = None
    snps = 0
    loci = 0
    for line in IN:
        if line[0] == "#":
             # Write header.
             OUT.write(line)
        else:
            snps += 1
            parts = line.split()
            locus = parts[0]
            if current_locus != locus:
                if current_locus is not None:
                    loci += 1
                    OUT.write(best_line)
                
                current_locus = locus
                best_NS = 0
                best_line = ""
            
            # Column 7 is INFO column of VCF file.
            NS = float(parts[7].split(";")[0].split("=")[1])
            if NS > best_NS:
                best_NS = NS
                best_line = line
            
    loci += 1
    OUT.write(best_line)
    IN.close()
    print("Total SNPS: " + str(snps) + "\nUnlinked SNPs: " + str(loci))
    OUT.close()


In [13]:
## Total, unfiltered loci. Not used in paper, so may skip if desired.
infile = "OL-c85t10-x45m75.recode.vcf"
outfile = "OL-c85t10-x45m75-full-u.vcf"
subsetSNPs(infile,outfile)

Total SNPS: 42118
Unlinked SNPs: 9845


In [15]:
## File without MAF filtering. May skip of desired
infile = "Inputs/OL-c85t10-x45m75-filt.recode.vcf"
outfile = "Inputs/OL-c85t10-x45m75-u.vcf"
subsetSNPs(infile,outfile)

Total SNPS: 41159
Unlinked SNPs: 9696


In [16]:
## File with MAF filtering 
## Creates file used in paper
infile = "Inputs/OL-c85t10-x45m75-maf025.recode.vcf"
outfile = "Inputs/OL-c85t10-x45m75-maf025-u.vcf"
subsetSNPs(infile,outfile)

Total SNPS: 13487
Unlinked SNPs: 6208


# Making outlier-only and neutral-only .vcf files
This section is only to be done after outlier loci have been identified. VCFtools is used to filter out outlier loci to create an outlier-only .vcf and a putative neutral-only .vcf.

### Intersect 2

In [12]:
#Make files of bad loci (that have at least one site with an outlier) to feed to vcftools. 
#IN = open('Outlier/x45m75maf025filt-pcaQ_OF_BS-isect2.snp', "r")
IN = open('Outlier/x45m75maf025filt-pcaQ_OF_BS10-isectUnion.snp',"r")
INL = open('Outlier/x45m75maf025filt-pcaQ_OF_BS10-isect2.loci',"r")
OUT = open('Making_Files/x45m75maf025filt-pcaQ_OF_BS10-isect2.badchrom', "w")
OUTg2 = open('Making_Files/x45m75maf025filt-pcaQ_OF_BS10-isect2.goodchrom', "w")
exset = set()
for line in INL:
    chrom = line.strip()
    if chrom not in exset:
        exset.add(chrom)
        OUT.write(" --not-chr locus_"+str(chrom))
print len(exset)
x = 0
for line in IN:
    chrom = line.strip().split("_")[1]
    snp = line.strip().split("_")[3]
    OUTg2.write("locus_"+chrom+"\t"+snp+"\n")
    x += 1
print x
OUT.close()
OUTg2.close()
IN.close()
INL.close()

129
235


In [13]:
%%sh
suffix=Inputs/OL-c85t10-x45m75-maf025
value=`cat Making_Files/x45m75maf025filt-pcaQ_OF_BS10-isect2.badchrom`
vcftools --vcf $suffix.recode.vcf --recode --recode-INFO-all $value \
--max-alleles 2 --min-alleles 2 --out $suffix-neutI2


VCFtools - 0.1.15
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf Inputs/OL-c85t10-x45m75-maf025.recode.vcf
	--not-chr locus_101035
	--not-chr locus_102819
	--not-chr locus_104611
	--not-chr locus_104742
	--not-chr locus_104832
	--not-chr locus_10670
	--not-chr locus_107169
	--not-chr locus_110252
	--not-chr locus_11196
	--not-chr locus_113174
	--not-chr locus_113647
	--not-chr locus_115948
	--not-chr locus_11774
	--not-chr locus_11905
	--not-chr locus_120275
	--not-chr locus_121489
	--not-chr locus_121492
	--not-chr locus_123004
	--not-chr locus_124257
	--not-chr locus_12991
	--not-chr locus_131325
	--not-chr locus_134081
	--not-chr locus_134388
	--not-chr locus_13818
	--not-chr locus_144194
	--not-chr locus_145020
	--not-chr locus_153852
	--not-chr locus_153863
	--not-chr locus_162959
	--not-chr locus_16337
	--not-chr locus_170867
	--not-chr locus_171395
	--not-chr locus_172232
	--not-chr locus_17308
	--not-chr locus_17359
	--not-chr locus_17888
	--not-c

Filters X SNPs, kept 13,136 SNPS 

In [14]:
%%sh
suffix=Inputs/OL-c85t10-x45m75-maf025
vcftools --vcf $suffix.recode.vcf --recode --recode-INFO-all --positions Making_Files/x45m75maf025filt-pcaQ_OF_BS10-isect2.goodchrom \
--max-alleles 2 --min-alleles 2 --out $suffix-outI2Union


VCFtools - 0.1.15
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf Inputs/OL-c85t10-x45m75-maf025.recode.vcf
	--recode-INFO-all
	--max-alleles 2
	--min-alleles 2
	--out Inputs/OL-c85t10-x45m75-maf025-outI2Union
	--positions Making_Files/x45m75maf025filt-pcaQ_OF_BS10-isect2.goodchrom
	--recode

After filtering, kept 117 out of 117 Individuals
Outputting VCF file...
After filtering, kept 235 out of a possible 13487 Sites
Run Time = 0.00 seconds


In [15]:
infile = "Inputs/OL-c85t10-x45m75-maf025-neutI2.recode.vcf"
outfile = "Inputs/OL-c85t10-x45m75-maf025-neutI2-u.vcf"
subsetSNPsM(infile,outfile)

Total SNPS: 13136
Unlinked SNPs: 6079


In [16]:
infile = "Inputs/OL-c85t10-x45m75-maf025-outI2Union.recode.vcf"
outfile = "Inputs/OL-c85t10-x45m75-maf025-outI2Union-u.vcf"
subsetSNPsM(infile,outfile)

Total SNPS: 235
Unlinked SNPs: 129
